In [1]:
%matplotlib inline
#from preamble import *
%load_ext watermark
%watermark -v -p sklearn,numpy,scipy,matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tools.plotting import scatter_matrix
import os
import psutil
process = psutil.Process(os.getpid())

#import seaborn as sns
#import seaborn as sns; sns.set(style="ticks", color_codes=True)
sns.set(style="ticks")
sns.set_style('whitegrid')
sns.set_color_codes()

CPython 3.5.5
IPython 5.1.0

sklearn 0.19.1
numpy 1.14.2
scipy 1.0.0
matplotlib 2.2.2


In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import pickle

from tqdm import tqdm_notebook
import numpy

fname = 'data/data.csv'
#fname = 'data/total_10to18.csv'
data_ = pd.read_csv(fname)
data = data_

In [3]:
data.shape

(11751, 73)

In [4]:
for idx, (h, a) in enumerate(zip(data['Home_Name'], data['Away_Name'])):
    if h == 'Real Valladolid':
        data.loc[idx, 'Home_Name'] = 'Valladolid'
    if a == 'Real Valladolid':
        data.loc[idx, 'Away_Name'] = 'Valladolid'
    if h == 'Deportivo Alaves':
        data.loc[idx, 'Home_Name'] = 'Alaves'
    if a == 'Deportivo Alaves':
        data.loc[idx, 'Away_Name'] = 'Alaves'
    if h == 'Parma Calcio 1913':
        data.loc[idx, 'Home_Name'] = 'Parma'
    if a == 'Parma Calcio 1913':
        data.loc[idx, 'Away_Name'] = 'Parma'

In [5]:
for idx, i in enumerate(data['Match_Date']):
    #if len(i) > 6:
    if type(i) != int and len(i) > 6:
        date = ''
        #if idx < 5:
        awer = i.split('-')
        date += awer[2]
        date += awer[1]
        date += awer[0].split(', ')[1]
        date = date.replace("Jan", '01')
        date = date.replace("Feb", '02')
        date = date.replace("Mar", '03')
        date = date.replace("Apr", '04')
        date = date.replace("May", '05')
        date = date.replace("Aug", '08')
        date = date.replace("Sep", '09')
        date = date.replace("Oct", '10')
        date = date.replace("Nov", '11')
        date = date.replace("Dec", '12')
        data.loc[idx,'Match_Date'] = date

In [6]:
data['Home_LP'] = data['Away_GF']
data['Away_LP'] = data['Home_GF']

In [7]:
%%time
match_results = []
match_results_text = []

for i in range(len(data)):
    if data.Home_GF[i] > data.Away_GF[i]:
        match_results.append(0)
        match_results_text.append('H')
    elif data.Home_GF[i] < data.Away_GF[i]:
        match_results.append(2)
        match_results_text.append('A')
    elif data.Home_GF[i] == data.Away_GF[i]:
        match_results.append(1)
        match_results_text.append('D')
    else:
        match_results_text.append('')

Wall time: 2.49 s


In [8]:
data['Result'] = match_results_text

In [9]:
with open('data/pickle/y_data.pkl', 'wb') as f:
    pickle.dump(match_results, f)

In [10]:
%%time
Team = data['Home_Name'].unique()
Winning_Streak = {}

for i in Team:
    Winning_Streak[i] = 0
    
HWS, AWS = [], []
for i in range(data.shape[0]):
    H = data.loc[i, 'Home_Name'] 
    A = data.loc[i, 'Away_Name']
    
    if Winning_Streak[H] >= 2:
        HWS.append(Winning_Streak[H])
    else:
        HWS.append(0)
        
    if Winning_Streak[A] >= 2:
        AWS.append(Winning_Streak[A])
    else:
        AWS.append(0)
        
    if data.loc[i, 'Home_GF'] > data.loc[i, 'Away_GF']:
        Winning_Streak[H] = Winning_Streak[H] + 1
    elif data.loc[i, 'Home_GF'] < data.loc[i, 'Away_GF']:
        Winning_Streak[A] = Winning_Streak[A] + 1
        
    if data.loc[i, 'Home_GF'] <= data.loc[i, 'Away_GF']:
        Winning_Streak[H] = 0
    elif data.loc[i, 'Home_GF'] >= data.loc[i, 'Away_GF']:
        Winning_Streak[A] = 0

data['HWS'] = HWS
data['AWS'] = AWS

Wall time: 3.25 s


In [11]:
Losing_Streak = {}

for i in Team:
    Losing_Streak[i] = 0
    
HLS, ALS = [], []
for i in range(data.shape[0]):
    H = data.loc[i, 'Home_Name'] 
    A = data.loc[i, 'Away_Name']
    
    if Losing_Streak[H] >= 2:
        HLS.append(Losing_Streak[H])
    else:
        HLS.append(0)
        
    if Losing_Streak[A] >= 2:
        ALS.append(Losing_Streak[A])
    else:
        ALS.append(0)
        
    if data.loc[i, 'Home_GF'] < data.loc[i, 'Away_GF']:
        Losing_Streak[H] = Losing_Streak[H] + 1
    elif data.loc[i, 'Home_GF'] > data.loc[i, 'Away_GF']:
        Losing_Streak[A] = Losing_Streak[A] + 1
        
    if data.loc[i, 'Home_GF'] >= data.loc[i, 'Away_GF']:
        Losing_Streak[H] = 0
    elif data.loc[i, 'Home_GF'] <= data.loc[i, 'Away_GF']:
        Losing_Streak[A] = 0

data['HLS'] = HLS
data['ALS'] = ALS

In [12]:
TeamValueCount = {}

for i in Team:
    TeamValueCount[i] = 0

In [13]:
with open('data/pickle/Totalteam_value.pkl', 'rb') as f:
    team_value = pickle.load(f)
with open('data/pickle/Italyteam_value.pkl', 'rb') as f:
    team_value2 = pickle.load(f)
with open('data/pickle/German_team_value.pkl', 'rb') as f:
    team_value3 = pickle.load(f)
team_value.update(team_value2)
team_value.update(team_value3)

In [14]:
team_value['Augsburg'] = team_value.pop('FC Augsburg')
team_value['Bayer Leverkusen'] = team_value.pop('Bay. Leverkusen')
team_value['Borussia Dortmund'] = team_value.pop('Bor. Dortmund')
team_value['Borussia M.Gladbach'] = team_value.pop('Bor. M\'gladbach')
team_value['Darmstadt'] = team_value.pop('SV Darmstadt 98')
team_value['Eintracht Braunschweig'] = team_value.pop('E. Braunschweig')
team_value['Eintracht Frankfurt'] = team_value.pop('E. Frankfurt')
team_value['FC Cologne'] = team_value.pop('1. FC Köln')
team_value['Fortuna Duesseldorf'] = team_value.pop('F. Düsseldorf')
team_value['Freiburg'] = team_value.pop('SC Freiburg')
team_value['Greuther Fuerth'] = team_value.pop('Greuther Fürth')
team_value['Hertha Berlin'] = team_value.pop('Hertha BSC')
team_value['Hoffenheim'] = team_value.pop('TSG Hoffenheim')
team_value['Ingolstadt'] = team_value.pop('FC Ingolstadt')
team_value['Kaiserslautern'] = team_value.pop('1.FC K\'lautern')
team_value['Mainz 05'] = team_value.pop('1.FSV Mainz 05')
team_value['Nuernberg'] = team_value.pop('1.FC Nuremberg')
team_value['Paderborn'] = team_value.pop('SC Paderborn')
team_value['RasenBallsport Leipzig'] = team_value.pop('RB Leipzig')
team_value['Schalke 04'] = team_value.pop('FC Schalke 04')
team_value['St. Pauli'] = team_value.pop('FC St. Pauli')
team_value['Wolfsburg'] = team_value.pop('VfL Wolfsburg')                                                   


                                               
team_value['Hull'] = team_value.pop('Hull City')
team_value['Manchester City'] = team_value.pop('Man City')
team_value['Manchester United'] = team_value.pop('Man Utd')
team_value['Newcastle United'] = team_value.pop('Newcastle')
team_value['Queens Park Rangers'] = team_value.pop('QPR')
team_value['Tottenham'] = team_value.pop('Spurs')
team_value['Stoke'] = team_value.pop('Stoke City')
team_value['West Bromwich Albion'] = team_value.pop('West Brom')
team_value['Wolverhampton Wanderers'] = team_value.pop('Wolves')


team_value['Alaves'] = team_value.pop('Alavés')
team_value['Atletico Madrid'] = team_value.pop('Atlético Madrid')
team_value['Osasuna'] = team_value.pop('CA Osasuna')
team_value['Leganes'] = team_value.pop('CD Leganés')
team_value['Celta Vigo'] = team_value.pop('Celta de Vigo')
team_value['Cordoba'] = team_value.pop('Córdoba CF')
team_value['Deportivo La Coruna'] = team_value.pop('Dep. La Coruña')
team_value['Elche'] = team_value.pop('Elche CF')
team_value['Barcelona'] = team_value.pop('FC Barcelona')
team_value['Getafe'] = team_value.pop('Getafe CF')
team_value['Girona'] = team_value.pop('Girona FC')
team_value['Granada'] = team_value.pop('Granada CF')
team_value['Hercules'] = team_value.pop('Hércules CF')
team_value['Levante'] = team_value.pop('Levante UD')
team_value['Malaga'] = team_value.pop('Málaga CF')
team_value['Mallorca'] = team_value.pop('RCD Mallorca')
team_value['Valladolid'] = team_value.pop('Real Valladolid')
team_value['Racing Santander'] = team_value.pop('Racing')
team_value['Eibar'] = team_value.pop('SD Eibar')
team_value['Sevilla'] = team_value.pop('Sevilla FC')
team_value['Sporting Gijon'] = team_value.pop('Sporting Gijón')
team_value['Almeria'] = team_value.pop('UD Almería')
team_value['Las Palmas'] = team_value.pop('UD Las Palmas')
team_value['Valencia'] = team_value.pop('Valencia CF')
team_value['Villarreal'] = team_value.pop('Villarreal CF')


team_value['Roma'] = team_value.pop('AS Roma')
team_value['Napoli'] = team_value.pop('SSC Napoli')
team_value['Udinese'] = team_value.pop('Udinese Calcio')
team_value['Chievo'] = team_value.pop('Chievo Verona')
team_value['Palermo'] = team_value.pop('US Palermo')
team_value['Cagliari'] = team_value.pop('Cagliari Calcio')
team_value['Robur Siena'] = team_value.pop('AC Siena')
team_value['Livorno'] = team_value.pop('AS Livorno')
team_value['Verona'] = team_value.pop('Hellas Verona')
team_value['Empoli'] = team_value.pop('FC Empoli')
team_value['SPAL 2013'] = team_value.pop('SPAL')
team_value['Carpi'] = team_value.pop('Carpi FC')

In [15]:
%%time

Home_Value = []
Away_Value = []

def incept_teamvalue(data):
    
    for i in range(len(data['Home_Name'])):
        Home = data['Home_Name'][i] 
        Away = data['Away_Name'][i]

        if data['League'][i] == 'BUNDES':
            match_round = 34
        else:
            match_round = 38
            
        if  Home in team_value:
            Home_Value.append(team_value[Home][0])
            TeamValueCount[Home] += 1 
        if  Away in team_value:
            Away_Value.append(team_value[Away][0])
            TeamValueCount[Away] += 1 
        if TeamValueCount[Home] == match_round:
            TeamValueCount[Home] = 0

        if TeamValueCount[Away] == match_round:
            TeamValueCount[Away] = 0
            del team_value[Away][0]
        
        if i != len(Home_Value) -1:
            print('확인 {}'.format(i))
incept_teamvalue(data)

Wall time: 1.7 s


In [16]:
data['Home_Value'] = Home_Value
data['Away_Value'] = Away_Value

In [17]:
season = []

for i in data['Match_Date']:
    i = int(i)
    if i < 110700:
        season.append('1011')
    elif i > 110700 and i < 120700:
        season.append('1112')
    elif i > 120700 and i < 130700:
        season.append('1213')
    elif i > 130700 and i < 140700:
        season.append('1314')
    elif i > 140700 and i < 150700:
        season.append('1415')
    elif i > 150700 and i < 160700:
        season.append('1516')
    elif i > 160700 and i < 170700:
        season.append('1617')
    elif i > 170700 and i < 180700:
        season.append('1718')
    elif i > 180700 and i < 190700:
        season.append('1819')

In [18]:
data['Season'] = season

In [19]:
for idx, i in enumerate(data['Home_Value']):

    if data['Home_Name'][idx] == 'Levante' and data['Season'][idx] == '1011':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Valladolid' and data['Season'][idx] == '1213':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Almeria' and data['Season'][idx] == '1415':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Eibar' and data['Season'][idx] == '1415':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Las Palmas' and data['Season'][idx] == '1516':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Osasuna' and data['Season'][idx] == '1617':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'St. Pauli' and data['Season'][idx] == '1011':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Fortuna Duesseldorf' and data['Season'][idx] == '1213':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Greuther Fuerth' and data['Season'][idx] == '1213':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Eintracht Braunschweig' and data['Season'][idx] == '1314':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Paderborn' and data['Season'][idx] == '1415':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Ingolstadt' and data['Season'][idx] == '1516':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Darmstadt' and data['Season'][idx] == '1516':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Darmstadt' and data['Season'][idx] == '1617':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Novara' and data['Season'][idx] == '1112':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Pescara' and data['Season'][idx] == '1213':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1    
    elif data['Home_Name'][idx] == 'Empoli' and data['Season'][idx] == '1415':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1      
    elif data['Home_Name'][idx] == 'Cesena' and data['Season'][idx] == '1415':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1  
    elif data['Home_Name'][idx] == 'Empoli' and data['Season'][idx] == '1516':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1       
    elif data['Home_Name'][idx] == 'Chievo' and data['Season'][idx] == '1617':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Palermo' and data['Season'][idx] == '1617':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Pescara' and data['Season'][idx] == '1617':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Crotone' and data['Season'][idx] == '1617':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Crotone' and data['Season'][idx] == '1718':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Blackpool' and data['Season'][idx] == '1011':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1
    elif data['Home_Name'][idx] == 'Norwich' and data['Season'][idx] == '1112':
        data.loc[idx, 'Home_Value'] = data.loc[idx, 'Home_Value'] * 0.1

In [20]:
"""Relative Home Value"""
relative_value = []

for i in range(data.shape[0]):
    #value_differ = data.loc[i, 'Home_Value'] - data.loc[i, 'Away_Value']
    value_differ = data.loc[i, 'Home_Value'] / (data.loc[i, 'Home_Value'] + data.loc[i, 'Away_Value'])
    
    #season = data.loc[i, 'Season']
    #league = data.loc[i, 'League']
    
    #average_value = data['Home_Value'][(data['Season'] == season) & (data['League'] == league)].mean()
    
    #value_differ = value_differ/average_value
    
    relative_value.append(value_differ)

In [21]:
data['RVD'] = relative_value
del data['Home_Value']
del data['Away_Value']

In [22]:
data.to_csv('data/중간저장.csv', index=False)

In [23]:
odds = pd.read_csv('data/Finalodds.csv')

In [24]:
for idx, i in enumerate(odds['home']):
    if odds.loc[idx, 'home'] == 'Sp Gijon':
        odds.loc[idx, 'home'] = 'Sporting Gijon'
    if odds.loc[idx, 'home'] == 'Betis':
        odds.loc[idx, 'home'] = 'Real Betis'
    if odds.loc[idx, 'home'] == 'RB Leipzig':
        odds.loc[idx, 'home'] = 'RasenBallsport Leipzig'
    if odds.loc[idx, 'home'] == 'Man United':
        odds.loc[idx, 'home'] = 'Manchester United'
    if odds.loc[idx, 'home'] == 'Braunschweig':
        odds.loc[idx, 'home'] = 'Eintracht Braunschweig'
    if odds.loc[idx, 'home'] == 'Ath Bilbao':
        odds.loc[idx, 'home'] = 'Athletic Bilbao'
    if odds.loc[idx, 'home'] == 'Ath Madrid':
        odds.loc[idx, 'home'] = 'Atletico Madrid'
    if odds.loc[idx, 'home'] == 'Man City':
        odds.loc[idx, 'home'] = 'Manchester City'
    if odds.loc[idx, 'home'] == 'Sociedad':
        odds.loc[idx, 'home'] = 'Real Sociedad'
    if odds.loc[idx, 'home'] == 'Stuttgart':
        odds.loc[idx, 'home'] = 'VfB Stuttgart'
    if odds.loc[idx, 'home'] == 'Santander':
        odds.loc[idx, 'home'] = 'Racing Santander'
    if odds.loc[idx, 'home'] == 'Wolves':
        odds.loc[idx, 'home'] = 'Wolverhampton Wanderers'
    if odds.loc[idx, 'home'] == 'Fortuna Dusseldorf':
        odds.loc[idx, 'home'] = 'Fortuna Duesseldorf'
    if odds.loc[idx, 'home'] == "M'gladbach":
        odds.loc[idx, 'home'] = 'Borussia M.Gladbach'
    if odds.loc[idx, 'home'] == 'Leverkusen':
        odds.loc[idx, 'home'] = 'Bayer Leverkusen'
    if odds.loc[idx, 'home'] == 'La Coruna':
        odds.loc[idx, 'home'] = 'Deportivo La Coruna'
    if odds.loc[idx, 'home'] == 'Hamburg':
        odds.loc[idx, 'home'] = 'Hamburger SV'
    if odds.loc[idx, 'home'] == 'Nurnberg':
        odds.loc[idx, 'home'] = 'Nuernberg'
    if odds.loc[idx, 'home'] == 'Ein Frankfurt':
        odds.loc[idx, 'home'] = 'Eintracht Frankfurt'
    if odds.loc[idx, 'home'] == 'Dortmund':
        odds.loc[idx, 'home'] = 'Borussia Dortmund'
    if odds.loc[idx, 'home'] == 'FC Koln':
        odds.loc[idx, 'home'] = 'FC Cologne'
    if odds.loc[idx, 'home'] == 'Espanol':
        odds.loc[idx, 'home'] = 'Espanyol'
    if odds.loc[idx, 'home'] == 'Hannover':
        odds.loc[idx, 'home'] = 'Hannover 96'
    if odds.loc[idx, 'home'] == 'Greuther Furth':
        odds.loc[idx, 'home'] = 'Greuther Fuerth'
    if odds.loc[idx, 'home'] == 'West Brom':
        odds.loc[idx, 'home'] = 'West Bromwich Albion'
    if odds.loc[idx, 'home'] == 'Vallecano':
        odds.loc[idx, 'home'] = 'Rayo Vallecano'
    if odds.loc[idx, 'home'] == 'Celta':
        odds.loc[idx, 'home'] = 'Celta Vigo'
    if odds.loc[idx, 'home'] == 'Hertha':
        odds.loc[idx, 'home'] = 'Hertha Berlin'
    if odds.loc[idx, 'home'] == 'QPR':
        odds.loc[idx, 'home'] = 'Queens Park Rangers'
    if odds.loc[idx, 'home'] == 'Mainz':
        odds.loc[idx, 'home'] = 'Mainz 05'
    if odds.loc[idx, 'home'] == 'Newcastle':
        odds.loc[idx, 'home'] = 'Newcastle United'
    if odds.loc[idx, 'home'] == 'St Pauli':
        odds.loc[idx, 'home'] = 'St. Pauli'
    if odds.loc[idx, 'home'] == 'Zaragoza':
        odds.loc[idx, 'home'] = 'Real Zaragoza'
    if odds.loc[idx, 'home'] == 'Milan':
        odds.loc[idx, 'home'] = 'AC Milan'
    if odds.loc[idx, 'home'] == 'Siena':
        odds.loc[idx, 'home'] = 'Robur Siena'
    if odds.loc[idx, 'home'] == 'Spal':
        odds.loc[idx, 'home'] = 'SPAL 2013'
        
        
for idx, i in enumerate(odds['away']):
    if odds.loc[idx, 'away'] == 'Sp Gijon':
        odds.loc[idx, 'away'] = 'Sporting Gijon'
    if odds.loc[idx, 'away'] == 'Betis':
        odds.loc[idx, 'away'] = 'Real Betis'
    if odds.loc[idx, 'away'] == 'RB Leipzig':
        odds.loc[idx, 'away'] = 'RasenBallsport Leipzig'
    if odds.loc[idx, 'away'] == 'Man United':
        odds.loc[idx, 'away'] = 'Manchester United'
    if odds.loc[idx, 'away'] == 'Braunschweig':
        odds.loc[idx, 'away'] = 'Eintracht Braunschweig'
    if odds.loc[idx, 'away'] == 'Ath Bilbao':
        odds.loc[idx, 'away'] = 'Athletic Bilbao'
    if odds.loc[idx, 'away'] == 'Ath Madrid':
        odds.loc[idx, 'away'] = 'Atletico Madrid'
    if odds.loc[idx, 'away'] == 'Man City':
        odds.loc[idx, 'away'] = 'Manchester City'
    if odds.loc[idx, 'away'] == 'Sociedad':
        odds.loc[idx, 'away'] = 'Real Sociedad'
    if odds.loc[idx, 'away'] == 'Stuttgart':
        odds.loc[idx, 'away'] = 'VfB Stuttgart'
    if odds.loc[idx, 'away'] == 'Santander':
        odds.loc[idx, 'away'] = 'Racing Santander'
    if odds.loc[idx, 'away'] == 'Wolves':
        odds.loc[idx, 'away'] = 'Wolverhampton Wanderers'
    if odds.loc[idx, 'away'] == 'Fortuna Dusseldorf':
        odds.loc[idx, 'away'] = 'Fortuna Duesseldorf'
    if odds.loc[idx, 'away'] == "M'gladbach":
        odds.loc[idx, 'away'] = 'Borussia M.Gladbach'
    if odds.loc[idx, 'away'] == 'Leverkusen':
        odds.loc[idx, 'away'] = 'Bayer Leverkusen'
    if odds.loc[idx, 'away'] == 'La Coruna':
        odds.loc[idx, 'away'] = 'Deportivo La Coruna'
    if odds.loc[idx, 'away'] == 'Hamburg':
        odds.loc[idx, 'away'] = 'Hamburger SV'
    if odds.loc[idx, 'away'] == 'Nurnberg':
        odds.loc[idx, 'away'] = 'Nuernberg'
    if odds.loc[idx, 'away'] == 'Ein Frankfurt':
        odds.loc[idx, 'away'] = 'Eintracht Frankfurt'
    if odds.loc[idx, 'away'] == 'Dortmund':
        odds.loc[idx, 'away'] = 'Borussia Dortmund'
    if odds.loc[idx, 'away'] == 'FC Koln':
        odds.loc[idx, 'away'] = 'FC Cologne'
    if odds.loc[idx, 'away'] == 'Espanol':
        odds.loc[idx, 'away'] = 'Espanyol'
    if odds.loc[idx, 'away'] == 'Hannover':
        odds.loc[idx, 'away'] = 'Hannover 96'
    if odds.loc[idx, 'away'] == 'Greuther Furth':
        odds.loc[idx, 'away'] = 'Greuther Fuerth'
    if odds.loc[idx, 'away'] == 'West Brom':
        odds.loc[idx, 'away'] = 'West Bromwich Albion'
    if odds.loc[idx, 'away'] == 'Vallecano':
        odds.loc[idx, 'away'] = 'Rayo Vallecano'
    if odds.loc[idx, 'away'] == 'Celta':
        odds.loc[idx, 'away'] = 'Celta Vigo'
    if odds.loc[idx, 'away'] == 'Hertha':
        odds.loc[idx, 'away'] = 'Hertha Berlin'
    if odds.loc[idx, 'away'] == 'QPR':
        odds.loc[idx, 'away'] = 'Queens Park Rangers'
    if odds.loc[idx, 'away'] == 'Mainz':
        odds.loc[idx, 'away'] = 'Mainz 05'
    if odds.loc[idx, 'away'] == 'Newcastle':
        odds.loc[idx, 'away'] = 'Newcastle United'
    if odds.loc[idx, 'away'] == 'St Pauli':
        odds.loc[idx, 'away'] = 'St. Pauli'
    if odds.loc[idx, 'away'] == 'Zaragoza':
        odds.loc[idx, 'away'] = 'Real Zaragoza'
    if odds.loc[idx, 'away'] == 'Milan':
        odds.loc[idx, 'away'] = 'AC Milan'
    if odds.loc[idx, 'away'] == 'Siena':
        odds.loc[idx, 'away'] = 'Robur Siena'
    if odds.loc[idx, 'away'] == 'Spal':
        odds.loc[idx, 'away'] = 'SPAL 2013'

In [25]:
for idx, (h, a) in enumerate(zip(odds['home'], odds['away'])):
    if h == 'Real Valladolid':
        odds.loc[idx, 'home'] = 'Valladolid'
    if a == 'Real Valladolid':
        odds.loc[idx, 'away'] = 'Valladolid'
    if h == 'Deportivo Alaves':
        odds.loc[idx, 'home'] = 'Alaves'
    if a == 'Deportivo Alaves':
        odds.loc[idx, 'away'] = 'Alaves'
    if h == 'Parma Calcio 1913':
        odds.loc[idx, 'home'] = 'Parma'
    if a == 'Parma Calcio 1913':
        odds.loc[idx, 'away'] = 'Parma'

In [26]:
h_odds = []
d_odds = []
a_odds = []
for idx, (x, y, z) in enumerate(zip(tqdm_notebook(data.Match_Date[:]), data.Home_Name[:], data.Away_Name[:])):
    if idx == 0:
        odds_table = pd.DataFrame(columns=['odds_w', 'odds_d', 'odds_l'])
    
    odd = odds[['odds_w', 'odds_d', 'odds_l']][(odds.date == int(x)) & (odds.home == y) & (odds.away == z)] 
    odds_table = pd.concat([odds_table, odd], axis=0)

In [27]:
odds_table = odds_table.reset_index(drop=True)

In [28]:
data = pd.concat([data, odds_table], axis=1)

In [29]:
data.to_csv('data/중간저장.csv', index=False)

In [30]:
def MPE(Goal, Lose, Match_num):
    if Goal > 0:
        mpe = (Goal**1.122777 / (Goal**1.072388 + Lose**1.127246)) * 2.49973 * Match_num
    else:
        mpe = (1 / (1 + Lose**1.127246)) * 2.49973 * Match_num
    return mpe

In [31]:
Recent5_TotalGoal = {}
Recent5_TotalLose = {}
TeamMatchCount = {}
Season = [1011, 1112, 1213, 1314, 1415, 1516, 1617, 1718, 1819]
for i in tqdm_notebook(Team):
    for season in Season:
        Goal = []
        Lose = []
        h = data[((data['Season']==str(season)) & (data['Home_Name']==i)) | ((data['Season']==str(season)) & (data['Away_Name']==i))]
        h = h.reset_index()
        season_team = str(season) + '_' + i
        for index in range(h.shape[0]):
            if h.loc[index, 'Home_Name'] == i:
                Goal.append(h.loc[index, 'Home_GF'])
                Lose.append(h.loc[index, 'Home_LP'])
            elif h.loc[index, 'Away_Name'] == i:
                Goal.append(h.loc[index, 'Away_GF'])
                Lose.append(h.loc[index, 'Away_LP'])
        if len(Goal) > 0:
            Recent5_TotalGoal[season_team] = Goal
            Recent5_TotalLose[season_team] = Lose
        TeamMatchCount[season_team] = 0

In [32]:
H_MPE5 = []
A_MPE5 = []
MPE5differ = []
for i in tqdm_notebook(range(data.shape[0])):
    season_h = str(data.loc[i, 'Season']) + '_' + data.loc[i, 'Home_Name']
    season_a = str(data.loc[i, 'Season']) + '_' + data.loc[i, 'Away_Name']
    
    if TeamMatchCount[season_h] >5:
        mpe_hg = sum(Recent5_TotalGoal[season_h][:5])
        mpe_hl = sum(Recent5_TotalLose[season_h][:5])
        
        mpe_h = MPE(mpe_hg, mpe_hl, 5)
        
        H_MPE5.append(mpe_h)
        
        del Recent5_TotalGoal[season_h][0]
        TeamMatchCount[season_h] += 1
    else:
        H_MPE5.append(0)
        TeamMatchCount[season_h] += 1
        
    if TeamMatchCount[season_a] >5:
        mpe_ag = sum(Recent5_TotalGoal[season_a][:5])
        mpe_al = sum(Recent5_TotalLose[season_a][:5])
        
        mpe_a = MPE(mpe_ag, mpe_al, 5)
        
        A_MPE5.append(mpe_a)
        
        del Recent5_TotalGoal[season_a][0]
        TeamMatchCount[season_a] += 1
    else:
        A_MPE5.append(0)
        TeamMatchCount[season_a] += 1

for hmpe, ampe in zip(H_MPE5, A_MPE5):
    if hmpe == 0 : hmpe = 0.1
    if ampe == 0 : ampe = 0.1
    relative = hmpe / (hmpe + ampe)
    MPE5differ.append(relative)

data['RMPE5'] = MPE5differ

In [33]:
data.to_csv('data/중간저장.csv', index=False)

In [34]:
#data = data.iloc[:, :83]

In [35]:
data = pd.read_csv('data/중간저장.csv')
data.head()

,League,MatchID,Match_Date,Home_Name,Away_Name,Home_GF,Away_GF,Home_LP,Away_LP,Result,...,HWS,AWS,HLS,ALS,Season,RVD,odds_w,odds_d,odds_l,RMPE5
0,LALIGA,433797,100828,Hercules,Athletic Bilbao,0.0,1.0,1.0,0.0,A,...,0,0,0,0,1011,0.369231,2.50,3.25,2.80,0.5
1,LALIGA,433804,100828,Malaga,Valencia,1.0,3.0,3.0,1.0,A,...,0,0,0,0,1011,0.285897,2.88,3.30,2.40,0.5
2,LALIGA,433808,100828,Levante,Sevilla,1.0,4.0,4.0,1.0,A,...,0,0,0,0,1011,0.113959,3.60,3.50,2.00,0.5
3,LALIGA,433809,100829,Espanyol,Getafe,3.0,1.0,1.0,3.0,H,...,0,0,0,0,1011,0.424757,2.10,3.30,3.50,0.5
4,LALIGA,433811,100829,Real Sociedad,Villarreal,1.0,0.0,0.0,1.0,H,...,0,0,0,0,1011,0.281145,3.00,3.25,2.38,0.5


In [36]:
feautres = [i for i in data.iloc[:,10:66].keys()]

In [ ]:
k = 3
for feature in tqdm_notebook(range(0, len(feautres), 2)):
    print(feautres[feature], feautres[feature+1])
    
    hRecent3_Stats = {}
    aRecent3_Stats = {}
    hTeamMatchCount = {}
    aTeamMatchCount = {}

    for i in Team:
        #hstat = []
        #astat = []
        hstat = list(data[feautres[feature]][data['Home_Name'] == i])
        astat = list(data[feautres[feature+1]][data['Away_Name'] == i])

        hRecent3_Stats[i] = hstat
        aRecent3_Stats[i] = astat
        hTeamMatchCount[i] = 1
        aTeamMatchCount[i] = 1

    r3stat = []
    #h3stat_ = 0
    #a3stat_ = 0
    for i in range(data.shape[0]):
        h3stat_ = 0
        a3stat_ = 0
        
        h = data.loc[i, 'Home_Name']
        a = data.loc[i, 'Away_Name']

        if hTeamMatchCount[h] > k:
            h3stat_ = sum(hRecent3_Stats[h][:k]) / k
            hTeamMatchCount[h] += 1
            del hRecent3_Stats[h][0]

        else:
            hTeamMatchCount[h] += 1

        if aTeamMatchCount[a] > k:
            a3stat_ = sum(aRecent3_Stats[a][:k]) / k
            aTeamMatchCount[a] += 1
            del aRecent3_Stats[a][0]
        else:
            aTeamMatchCount[a] += 1

        if h3stat_ == 0:h3stat_ = 0.001
        if a3stat_ == 0:a3stat_ = 0.001
        temp = h3stat_ / (h3stat_ + a3stat_)
        #temp = h3stat_ - a3stat_
        r3stat.append(temp)
    #print(data.shape, len(r3stat))
    feature_name = feautres[feature][5:]
    data[feature_name + '_{}'.format(k)] = r3stat

In [39]:
data.tail()

,League,MatchID,Match_Date,Home_Name,Away_Name,Home_GF,Away_GF,Home_LP,Away_LP,Result,...,Away_Tackles_3,RATE_3,Home_RATE_3,Away_RATE_3,Touch_3,Home_Touch_3,Away_Touch_3,Save_ratio_3,Home_Save_ratio_3,Away_Save_ratio_3
11746,BUNDES,1294579,180922,Wolfsburg,Freiburg,NaN,NaN,NaN,NaN,NaN,...,22.333333,0.515598,6.886667,6.470000,0.524338,614.000000,557.000000,0.570271,0.470833,0.354798
11747,BUNDES,1294580,180922,Nuernberg,Hannover 96,NaN,NaN,NaN,NaN,NaN,...,16.333333,0.496484,6.590000,6.683333,0.470358,600.333333,676.000000,0.282908,0.177778,0.450617
11748,BUNDES,1294572,180922,Schalke 04,Bayern Munich,NaN,NaN,NaN,NaN,NaN,...,14.333333,0.476534,6.600000,7.250000,0.391798,563.666667,875.000000,0.407036,0.282828,0.412020
11749,BUNDES,1294574,180923,Bayer Leverkusen,Mainz 05,NaN,NaN,NaN,NaN,NaN,...,17.666667,0.500625,6.673333,6.656667,0.541287,692.666667,587.000000,0.359684,0.224691,0.400000
11750,BUNDES,1294576,180923,Eintracht Frankfurt,RasenBallsport Leipzig,NaN,NaN,NaN,NaN,NaN,...,21.666667,0.499750,6.666667,6.673333,0.474399,559.000000,619.333333,0.432747,0.260652,0.341667


In [41]:
data.to_csv('data/중간저장.csv', index=False)

In [ ]:
data2 = data

In [ ]:
"""
#시즌 초 경기 삭제 (5경기 미만)
for i in range(data2.shape[0]):
    date = str(data2.loc[i, 'Match_Date'])[-4:]
    date = int(date)
    if data2.loc[i, 'RMPE5'] == 0.5 and date < 1020:
        data2 = data2.drop(i, 0)
data2 = data2.reset_index(drop=True)
"""

In [43]:
match_table = data2.iloc[:, :10]
match_table['Season'] = data2['Season']
match_table.drop(['Home_LP', 'Away_LP'], axis=1, inplace=True)
match_table

,League,MatchID,Match_Date,Home_Name,Away_Name,Home_GF,Away_GF,Result,Season
0,LALIGA,433797,100828,Hercules,Athletic Bilbao,0.0,1.0,A,1011
1,LALIGA,433804,100828,Malaga,Valencia,1.0,3.0,A,1011
2,LALIGA,433808,100828,Levante,Sevilla,1.0,4.0,A,1011
3,LALIGA,433809,100829,Espanyol,Getafe,3.0,1.0,H,1011
4,LALIGA,433811,100829,Real Sociedad,Villarreal,1.0,0.0,H,1011
5,LALIGA,433819,100829,Osasuna,Almeria,0.0,0.0,D,1011
6,LALIGA,433807,100829,Deportivo La Coruna,Real Zaragoza,0.0,0.0,D,1011
7,LALIGA,433803,100829,Racing Santander,Barcelona,0.0,3.0,A,1011
8,LALIGA,433822,100829,Mallorca,Real Madrid,0.0,0.0,D,1011
9,LALIGA,433794,100830,Atletico Madrid,Sporting Gijon,4.0,0.0,H,1011


In [44]:
prediction_table = data2.iloc[:,66:]
del prediction_table['Season']

In [45]:
import numpy
count = 0
for i in tqdm_notebook(prediction_table.columns):
    for idx, j in enumerate(prediction_table[i]):
        if numpy.isnan(j) == True:
            count += 1
            print('{},{} 에서 nan 검출'.format(i,idx))
            
print(count)


0


In [46]:
prediction_table = pd.concat([match_table, prediction_table], axis=1)
prediction_table

,League,MatchID,Match_Date,Home_Name,Away_Name,Home_GF,Away_GF,Result,Season,Home_IAR,...,Away_Tackles_3,RATE_3,Home_RATE_3,Away_RATE_3,Touch_3,Home_Touch_3,Away_Touch_3,Save_ratio_3,Home_Save_ratio_3,Away_Save_ratio_3
0,LALIGA,433797,100828,Hercules,Athletic Bilbao,0.0,1.0,A,1011,6.966667,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000
1,LALIGA,433804,100828,Malaga,Valencia,1.0,3.0,A,1011,6.100000,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000
2,LALIGA,433808,100828,Levante,Sevilla,1.0,4.0,A,1011,0.000000,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000
3,LALIGA,433809,100829,Espanyol,Getafe,3.0,1.0,H,1011,6.636667,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000
4,LALIGA,433811,100829,Real Sociedad,Villarreal,1.0,0.0,H,1011,6.390000,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000
5,LALIGA,433819,100829,Osasuna,Almeria,0.0,0.0,D,1011,0.000000,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000
6,LALIGA,433807,100829,Deportivo La Coruna,Real Zaragoza,0.0,0.0,D,1011,6.500000,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000
7,LALIGA,433803,100829,Racing Santander,Barcelona,0.0,3.0,A,1011,6.370000,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000
8,LALIGA,433822,100829,Mallorca,Real Madrid,0.0,0.0,D,1011,0.000000,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000
9,LALIGA,433794,100830,Atletico Madrid,Sporting Gijon,4.0,0.0,H,1011,0.000000,...,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000,0.500000,0.001000,0.001000


In [47]:
prediction_table.to_csv('data/prediction_data(DL).csv', index=False)